In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# FE621 Homework 1
Cian Gahan

**<span style="color: red">NOTE</span>:** When setting up and running my data gathering script, I misread the assignment instructions and downloaded option/price data for the SPX index rather than the SPY ETF. I did not realize this until after Thursday's trading hours and therefore could not switch out the data for the correct ticker. I anticipate the results will be very similar. Vol/open interest relative to the contract size being 10x as much seems in line with SPY on yahoo finance when I checked, potentially smaller. I would anticipate liquidity is at worst similar since the granularity of contracts is higher on SPX (every 5 index points at the money, where SPY trades around 1/10th the value and has every $1-spaced options at the money), and SPY mostly does a good job tracking SPX so I don't think this will have a major impact on analysis or results.

Moving forward in the assignment I will use SPX in place of SPY.

## Part 1: Data Gathering Component
### 1.1: Data Gathering Function & Bonus
- See ```scripts/hw1_yf_ingest.py``` for raw ingestion
- See ```scripts/hw1_yf_ingest_cleaner.py``` for cleaning/combination
### 1.2:
- Why do additional maturities exist? Given that TSLA, SPX, and VIX are all highly liquid, popular tickers with a lot of speculative interest, traders have demand for products that speculate on volatility/short term movements with finer control over the time window than the traditional monthly schedule, especially as the maturity date approaches. Therefore exchanges/market makers introduce options with more maturities as these dates approach, meeting demand without fragmenting long-term maturity liquidity (eg a month or two out, weekly options may be introduced, and a week or so out, daily options may be introduced in the case of SPX).

In [6]:
# load in current prices
current_prices = pd.read_csv("../data/cleaned/current_prices.csv", index_col=0)
current_prices.index = pd.to_datetime(current_prices.index)
print(current_prices.head())

# load in options
options = pd.read_csv("../data/cleaned/options.csv", index_col=0)
print(options.head())

                  Close ticker  fed_funds
2026-02-12   415.815002   TSLA     0.0364
2026-02-13   419.630402   TSLA     0.0364
2026-02-12  6861.810059   ^SPX     0.0364
2026-02-13  6873.540039   ^SPX     0.0364
2026-02-12    19.879999   ^VIX     0.0364
       contractSymbol  strike   bid   ask optionType  expiration underlying  \
0  VIX260218C00010000    10.0  9.75  10.4       call  2026-02-18       ^VIX   
1  VIX260218C00010500    10.5  9.25   9.9       call  2026-02-18       ^VIX   
2  VIX260218C00011000    11.0  8.75   9.4       call  2026-02-18       ^VIX   
3  VIX260218C00011500    11.5  8.25   8.9       call  2026-02-18       ^VIX   
4  VIX260218C00012000    12.0  7.75   8.4       call  2026-02-18       ^VIX   

    data_date  
0  2026-02-12  
1  2026-02-12  
2  2026-02-12  
3  2026-02-12  
4  2026-02-12  


### 1.3:
- SPX: Index published by S&P consisting of a market-cap weighted average of ~500 of the largest companies trading in the US stock market. Purpose is to provide a measure of overall US stock market performance
- SPY: ETF that tracks the SPX index, managed by State Street. Purpose is to gain easy exposure to the entire US stock market
- VIX: Index published by CBOE that estimates 30 day market-implied volatility using a weighted combination of implied volatilities calculated from out-of-the-money SPX call and put options between 23 and 37 days to maturity. Purpose is to provide a measure for expected volatility and allow for trading/speculation on it directly
- TSLA: Tesla Stock. Purpose is to invest in Tesla
- Example option symbol decomposition: 
    * TICKERYYMMDD{C/P}STRIKE (strike listed as XXXXX.XXX fixed-point)
    * eg TSLA260220C00450000 - tesla call maturing on Feb 20th with strike $450.
    * Note on SPX options: Options may be listed with tickers as SPX or SPXW in raw data. SPXW are typically found in the finer-grain weekly maturities but may also exist at certain prices in the traditional maturity dates. The mechanical difference is that SPXW expire/are settled using VWAP near the end of the trading day on Friday, while SPX expire at the opening auction on Friday. To simplify analysis I removed this distinction while cleaning the data

### 1.4:
- For short term interest rate I used the Fed Funds (effective) rate from the Fed website which was 3.64% for both days in my data